# Linear regression

[![Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/lento234/ml-tutorials/blob/main/01-basics/01-linear_regression.ipynb)

**References**:
- https://pytorch.org/
- https://github.com/yunjey/pytorch-tutorial/blob/master/tutorials/01-basics/linear_regression/main.py
- https://en.wikipedia.org/wiki/Linear_regression
- https://en.wikipedia.org/wiki/Backpropagation
- https://en.wikipedia.org/wiki/Stochastic_gradient_descent



**Linear regression**

Given a data set $\{y_i, x_i\}_{i=1}^n$ of $n$ statistical units, we have the following relationship:

$$ y_i = \beta_1 x_i + \beta_0 + \varepsilon_i, \qquad i=1,\ldots, n,$$


where:
- $y_i$ is observed value (dependent variable
- $x_i$ is the independent variable
- $\beta_1$ and $\beta_0$ are the trainable parameters
- $\varepsilon_i$ is the measurement error

**Table of Content**
1. [Setup environment](#setup)
2. [Define the model, loss function and optimizer](#model)
3. [Train the model](#train)
4. [Predict and evaluate the model](#evaluate)
5. [Save the trained model](#save)
6. [Load pretrained model](#load)

<a id="setup"></a>
## 1. Setup environment

### Load packages / modules

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim

In [ ]:
mpl.style.use('seaborn-poster')
mpl.rcParams['mathtext.fontset'] = 'cm'
mpl.rcParams['figure.figsize'] = 5 * np.array([1.618033988749895, 1])

In [ ]:
# Reproducability
seed = 234
np.random.seed(seed)
torch.random.manual_seed(seed);

### Generate training dataset

In [ ]:
n = 500
beta_1 = 2.0
beta_0 = 0.3
eps_std = 0.2

In [ ]:
x_train = np.random.rand(n,1)-0.5
eps = np.random.normal(scale=eps_std, size=(n,1))
y_train = x_train*beta_1 + beta_0 + eps

In [ ]:
fig, ax = plt.subplots()
ax.plot(x_train, y_train, 'k.', label='observations')
ax.plot(x_train, x_train*beta_1 + beta_0, 'tab:gray',
        label=r'analytical ($\beta_1={:.2f}$, $\beta_0={:.2f}$)'.format(beta_1, beta_0))
ax.set(xlabel='$x$', ylabel='$y$', xlim=(-0.52,0.52), ylim=(-1.0, 1.5));
# ax.legend();
fig.legend(loc='right', bbox_to_anchor=(1.5, 0.6));

<a id="model"></a>
## 2. Define the model, loss function and optimizer

### Hyperparameters

In [ ]:
n_features = 1 # number of nodes (1 weight + 1 bias)
num_epochs = 150 # i.e. number of iterations
learning_rate = 0.5

### Model

$$ \mathcal{F}: x \rightarrow y $$

**Linear 1-D model**:

$$ \mathcal{F}(x) = \hat{y} = x a + b$$

where:
- $y$ is the ground truth
- $\hat{y}$ is the predicted output
- $a$ is the learnable weight, i.e. $a \equiv \beta_1$
- $b$ is the learnable bias, i.e. $b \equiv \beta_0$

In [ ]:
# Linear regression model
model = nn.Linear(in_features=n_features, out_features=n_features)

### Loss function

**$L^1$-norm:**

$$ \mathcal{L}(X, Y) = \textrm{mean} \left( \{l_1,\dots,l_N\}^\top \right), \quad l_n = \left| \hat{y}_n - y_n \right| $$
where:
- $X \in \mathbb{R}^N$ is the input matrix
- $Y \in \mathbb{R}^N$ is the ground truth matrix
- $N$ is the batch size (for now, batch size = number of examples)

In [ ]:
# Loss and optimizer
criterion = nn.L1Loss() # L^1-norm, aka. mean absolute error loss

### Optimizer

**Stochastic gradient descent**

$$ \theta_{n+1} = \theta_n - \eta \nabla \mathcal{L}(\theta_n) $$

where:
- $\theta$ are the trainable parameters
- $\eta$ is the learning rate (i.e. step size)

In [ ]:
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate) # Stochastic gradient-descent

<a id="train"></a>
## 3. Train the model

$$ \mathcal{F}^* = \arg \min_{\mathcal{F}}\ \mathcal{L}(X, Y)$$

**Algorithm:**
1. Convert data to torch tensor: `torch.from_numpy(...)`
2. Forward pass: `y_pred = model(x)`
3. Calculate loss: `loss = criterion(y_pred, y)`
4. Compute gradients: `loss.backward()`
5. Update weights: `optimizer.step()`

In [ ]:
history = dict(loss=[], parameters=[])
for epoch in range(num_epochs):
    # 1. Convert numpy arrays to torch tensors
    x = torch.from_numpy(x_train.astype('float32'))
    y = torch.from_numpy(y_train.astype('float32'))

    # 2. Forward pass
    y_pred = model(x) # prediction step
    
    # 3. Calculate loss
    loss = criterion(y_pred, y)
    
    # 4. Backward propagation 
    optimizer.zero_grad() # reset gradients to zero
    loss.backward() # backprop: calculate gradients w.r.t to loss
    
    # 5. Update weights
    optimizer.step() # update gradient
    
    # 6. Log
    history['loss'].append(loss.item())
    history['parameters'].append([param.detach().item() for param in model.parameters()])
    if (epoch % 10) == 0 or epoch==(num_epochs-1):
        print ('[Epoch {}]: loss = {:.4f}'.format(epoch, loss.item()))
        

### Training loss history

In [ ]:
fig, ax = plt.subplots()
ax.plot(history['loss'], '.-')
ax.set(xlabel='epoch', ylabel='MSE loss',
       title='Training loss history');

<a id="evaluate"></a>
## 4. Predict and evaluate the model

### Training accuracy

In [ ]:
# Predict
predict = model(torch.from_numpy(x_train.astype('float32'))).detach().numpy()

# Plot the graph
fig, ax = plt.subplots()
ax.plot(x_train, y_train, 'k.', label='observations')
ax.plot(x_train, x_train*beta_1 + beta_0, 'tab:gray', lw=5,
        label=r'analytical: ($\beta_1={:.2f}$, $\beta_0={:.2f}$)'.format(beta_1, beta_0))
ax.plot(x_train, x_train*history['parameters'][0][0] + history['parameters'][0][1], 'tab:blue',
        label=r'prediction [epoch 0]: ($\beta_1={:.2f}$, $\beta_0={:.2f}$)'.format(*history['parameters'][0]))
ax.plot(x_train, predict, 'tab:red',
        label=r'prediction [trained]: ($\beta_1={:.2f}$, $\beta_0={:.2f}$)'.format(*history['parameters'][-1]))
ax.set(xlabel='$x$', ylabel='$y$',
       title='Training accuracy',
       xlim=(-0.52,0.52), ylim=(-1.0, 1.5))

fig.legend(loc='right', bbox_to_anchor=(1.6, 0.6));

### Optimization over the loss landscape

In [ ]:
# L^1 norm: numpy version
l1_norm = lambda y_hat, y: np.abs(y_hat - y).mean(axis=0)

# Hypothesis space
beta_1_h, beta_0_h = np.meshgrid(np.linspace(0, 4),
                                 np.linspace(-0.2, 0.8))
obj_surf = l1_norm(x_train[...,None]*beta_1_h[None] + beta_0_h[None],
                   y_train[...,None])

In [ ]:
fig, ax = plt.subplots()
im = ax.contourf(beta_1_h, beta_0_h, obj_surf)
ax.plot(*history['parameters'][0], 'c.', ms=15, label='start', zorder=10)
ax.plot(beta_1, beta_0, 'r.', ms=20, label='optima', zorder=10)
ax.plot(np.array(history['parameters'])[:,0], np.array(history['parameters'])[:,1],
        '.-', c='k', label='path')
ax.set(xlabel=r'$\beta_1$ (slope)', ylabel=r'$\beta_0$ (offset)')
fig.colorbar(im, ax=ax)
fig.legend(loc='right', bbox_to_anchor=(1.15, 0.7));

<a id="save"></a>
## 5. Save the trained model

In [ ]:
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

<a id="load"></a>
## 6. Load pretrained model

In [ ]:
checkpoint = torch.load('model.ckpt')
checkpoint

In [ ]:
model.load_state_dict(checkpoint)